In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
from datetime import datetime, timedelta
from time import sleep

import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()

import pandas as pd

# 지역코드 api 불러오기

In [4]:
import requests

# API 정보 입력
API_KEY = "20250525142802gisnv918t6t9ljuirpt99a0icb"

#시도
def get_sd_list():
    url = f"https://stcis.go.kr/openapi/areacode.json?apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data.get('status') != 'OK':
        print("시도 API 호출 실패:", data)
        return pd.DataFrame()
    return pd.DataFrame(data.get('result', []))

sd_df = get_sd_list()
sd_df.head()

,sdCd,sdNm,sggCd,sggNm,emdCd,emdNm
0,11,서울특별시,None,None,None,None
1,21,부산직할시,None,None,None,None
2,22,대구직할시,None,None,None,None
3,23,인천직할시,None,None,None,None
4,24,광주직할시,None,None,None,None


In [5]:
#시군구
def get_sgg_list(sdCd):
    url = f"https://stcis.go.kr/openapi/areacode.json?apikey={API_KEY}&sdCd={sdCd}"
    response = requests.get(url)
    data = response.json()
    if data.get('status') != 'OK':
        print(f"시군구 API 호출 실패: sdCd={sdCd}, 응답:", data)
        return pd.DataFrame()
    return pd.DataFrame(data.get('result', []))


all_sgg_df = pd.DataFrame()

for i in range(len(sd_df)):
    sdCd = sd_df['sdCd'].iloc[i]
    sgg_df = get_sgg_list(sdCd)
    print(f"sdCd={sdCd} 의 시군구 개수: {len(sgg_df)}")
    print(sgg_df.head())
    all_sgg_df = pd.concat([all_sgg_df, sgg_df], ignore_index=True)

# 확인
all_sgg_df.head()

sdCd=11 의 시군구 개수: 25
  sdCd   sdNm  sggCd sggNm emdCd emdNm
0   11  서울특별시  11110   종로구  None  None
1   11  서울특별시  11140    중구  None  None
2   11  서울특별시  11170   용산구  None  None
3   11  서울특별시  11200   성동구  None  None
4   11  서울특별시  11215   광진구  None  None
sdCd=21 의 시군구 개수: 12
  sdCd   sdNm  sggCd sggNm emdCd emdNm
0   21  부산직할시  21110    중구  None  None
1   21  부산직할시  21140    서구  None  None
2   21  부산직할시  21170    동구  None  None
3   21  부산직할시  21200   영도구  None  None
4   21  부산직할시  21230  부산진구  None  None
sdCd=22 의 시군구 개수: 7
  sdCd   sdNm  sggCd sggNm emdCd emdNm
0   22  대구직할시  22110    중구  None  None
1   22  대구직할시  22140    동구  None  None
2   22  대구직할시  22170    서구  None  None
3   22  대구직할시  22200    남구  None  None
4   22  대구직할시  22230    북구  None  None
sdCd=23 의 시군구 개수: 6
  sdCd   sdNm  sggCd sggNm emdCd emdNm
0   23  인천직할시  23110    중구  None  None
1   23  인천직할시  23140    동구  None  None
2   23  인천직할시  23170    남구  None  None
3   23  인천직할시  23200   남동구  None  None
4   23  인천직할시  23230 

,sdCd,sdNm,sggCd,sggNm,emdCd,emdNm
0,11,서울특별시,11110,종로구,None,None
1,11,서울특별시,11140,중구,None,None
2,11,서울특별시,11170,용산구,None,None
3,11,서울특별시,11200,성동구,None,None
4,11,서울특별시,11215,광진구,None,None


In [ ]:
#읍면동
def get_emd_list(sdCd, sggCd):
    url = f"https://stcis.go.kr/openapi/areacode.json?apikey={API_KEY}&sdCd={sdCd}&sggCd={sggCd}"
    response = requests.get(url)
    data = response.json()
    if data.get('status') != 'OK':
        print(f"읍면동 API 호출 실패: sdCd={sdCd}, sggCd={sggCd}, 응답:", data)
        return pd.DataFrame()
    return pd.DataFrame(data.get('result', []))

all_emd_df = pd.DataFrame()

for i in range(len(all_sgg_df)):
    sdCd = all_sgg_df['sdCd'].iloc[i]
    sggCd = all_sgg_df['sggCd'].iloc[i]
    emd_df = get_emd_list(sdCd, sggCd)
    all_emd_df = pd.concat([all_emd_df, emd_df], ignore_index=True)
    sleep(0.5)

# 확인
all_emd_df.head()

In [13]:
all_emd_df.to_csv('data/id_df.csv', index=False, encoding='utf-8-sig')

# 추출 희망 지역 출발지와 도착지의 코드 리스트 추출

In [2]:
# id_df.csv 파일을 불러와서 지역 코드 데이터프레임 생성
all_emd_df = pd.read_csv('data/id_df.csv', encoding='utf-8-sig')

In [3]:
# 서울시의 모든 읍면동 코드 추출(출발지)
seoul_cd_list = list(all_emd_df.loc[all_emd_df['sdNm']=='서울특별시']['emdCd'])

# 서울시의 핫플레이스 지역 코드 추출(도착지)
hotplace_list = ['도곡동','삼성동','능동','신촌동','서초동','이화동','역삼동','서교동','수서동']
hot_seoul_cd_list = list(all_emd_df.loc[all_emd_df['sdNm']=='서울특별시'].loc[all_emd_df['emdNm'].isin(hotplace_list)].drop(index=[253,958,259,301,343,821,1028,965,246,266,250], axis=0)['emdCd'])

In [10]:
all_emd_df.loc[all_emd_df['sdNm']=='서울특별시'].loc[all_emd_df['emdNm'].isin(hotplace_list)].drop(index=[253,958,259,301,343,821,1028,965,246,266,250], axis=0)


,sdCd,sdNm,sggCd,sggNm,emdCd,emdNm
64,11,서울특별시,11110,종로구,1111016500,이화동
328,11,서울특별시,11215,광진구,1121510200,능동
634,11,서울특별시,11410,서대문구,1141011400,신촌동
682,11,서울특별시,11440,마포구,1144012000,서교동
961,11,서울특별시,11650,서초구,1165010800,서초동
1013,11,서울특별시,11680,강남구,1168010100,역삼동
1017,11,서울특별시,11680,강남구,1168010500,삼성동
1026,11,서울특별시,11680,강남구,1168011500,수서동
1029,11,서울특별시,11680,강남구,1168011800,도곡동


In [13]:
all_emd_df.loc[all_emd_df['sdNm']=='서울특별시'].loc[all_emd_df['emdNm'].isin(hotplace_list)]


,sdCd,sdNm,sggCd,sggNm,emdCd,emdNm
64,11,서울특별시,11110,종로구,1111016500,이화동
246,11,서울특별시,11200,성동구,1120002200,수서동
250,11,서울특별시,11200,성동구,1120002600,역삼동
253,11,서울특별시,11200,성동구,1120002900,도곡동
259,11,서울특별시,11200,성동구,1120003500,삼성동
266,11,서울특별시,11200,성동구,1120004200,서초동
301,11,서울특별시,11200,성동구,1120012100,능동
328,11,서울특별시,11215,광진구,1121510200,능동
343,11,서울특별시,11230,동대문구,1123000800,능동
634,11,서울특별시,11410,서대문구,1141011400,신촌동


# 교통정보 api 불러오기

## 원래 불러오는 방식: 사용 하지 않기로 결정함

In [ ]:
import requests
import json
from datetime import datetime, timedelta

# API 키 설정
API_KEY = "20250516163905aa0363qf6pse5eedgu7j5n111m"

# 날짜 범위 설정
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 1)

# 결과를 저장할 리스트 초기화
all_results = []

current_date = start_date
while current_date <= end_date: # 날짜 범위에 따라 API 호출
    for stgEmdCd in seoul_cd_list:
        for arrEmdCd in hot_seoul_cd_list:    # 서울시의 읍면동 코드와 핫플레이스 지역 코드 조합하여 API 호출
            opratDate = current_date.strftime('%Y%m%d')
            url = (
                f"https://stcis.go.kr/openapi/quarterod.json"
                f"?apikey={API_KEY}"
                f"&opratDate={opratDate}"
                f"&stgEmdCd={stgEmdCd}"
                f"&arrEmdCd={arrEmdCd}"
            )
            response = requests.get(url)
            data = json.loads(response.text)
            if 'result' in data:
                all_results.extend(data['result'])
            current_date += timedelta(days=1)


## 더 빠르게 불러오는 방식(by gpt): 이 방식으로 결정

### 정상 응답 확인용 코드

In [ ]:
import aiohttp
import asyncio

API_KEY = "20250516163905aa0363qf6pse5eedgu7j5n111m"
opratDate = "20230101"
stgEmdCd = "1111010100"
arrEmdCd = "1141011400"

async def test_fetch():
    url = (
        f"https://stcis.go.kr/openapi/quarterod.json"
        f"?apikey={API_KEY}"
        f"&opratDate={opratDate}"
        f"&stgEmdCd={stgEmdCd}"
        f"&arrEmdCd={arrEmdCd}"
    )
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print("Status:", response.status)
            print("Content-Type:", response.headers.get('Content-Type'))
            text = await response.text()
            print(text[:500])  # 응답 일부 출력

asyncio.run(test_fetch())


Status: 502
Content-Type: text/html; charset=iso-8859-1
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>502 Bad Gateway</title>
</head><body>
<h1>Bad Gateway</h1>
<p>The proxy server received an invalid
response from an upstream server.<br />
</p>
</body></html>



### 메인코드1

In [ ]:
################ 수정 구간 ################
API_KEY = "20250516163905aa0363qf6pse5eedgu7j5n111m" # 개인 API 키로 변경
start_date = datetime(2023, 1, 1) # 시작 날짜
end_date = datetime(2023, 1, 1) # 종료 날짜
################ 수정 구간 ################
all_results = []

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            return data.get('result', [])
        else:
            print(f"Error: {response.status} / {url}")
            return []

async def main():
    tasks = []
    current_date = start_date
    async with aiohttp.ClientSession() as session:
        while current_date <= end_date:
            opratDate = current_date.strftime('%Y%m%d')
            # 1. 출발지: 서울시, 도착지: 핫플레이스
            for stgEmdCd in seoul_cd_list: # 서울시의 읍면동 코드
                for arrEmdCd in hot_seoul_cd_list: # 핫플레이스 지역 코드
                    if stgEmdCd == arrEmdCd: # 같은 지역은 제외
                        continue
                    url = (
                        f"https://stcis.go.kr/openapi/quarterod.json"
                        f"?apikey={API_KEY}"
                        f"&opratDate={opratDate}"
                        f"&stgEmdCd={stgEmdCd}"
                        f"&arrEmdCd={arrEmdCd}"
                    )
                    tasks.append(fetch(session, url))
            # 2. 출발지: 핫플레이스, 도착지: 서울시
            for stgEmdCd in hot_seoul_cd_list: # 핫플레이스 지역 코드
                for arrEmdCd in seoul_cd_list: # 서울시의 읍면동 코드
                    if stgEmdCd == arrEmdCd: # 같은 지역은 제외
                        continue
                    url = (
                        f"https://stcis.go.kr/openapi/quarterod.json"
                        f"?apikey={API_KEY}"
                        f"&opratDate={opratDate}"
                        f"&stgEmdCd={stgEmdCd}"
                        f"&arrEmdCd={arrEmdCd}"
                    )
                    tasks.append(fetch(session, url))
            current_date += timedelta(days=1)
        results = await asyncio.gather(*tasks)
        for result in results:
            all_results.extend(result)

# Jupyter 환경에서는 await main()
await main()
print(f"총 데이터 건수: {len(all_results)}")

In [103]:
df = pd.DataFrame(all_results)
df

,opratDate,stgSdCd,stgSdNm,stgSggCd,stgSggNm,stgEmdCd,stgEmdNm,arrSdCd,arrSdNm,arrSggCd,arrSggNm,arrEmdCd,arrEmdNm,tzon,quater,useStf,useTm
0,20230101,11,서울특별시,11110,종로구,1111010100,청운동,11,서울특별시,11410,서대문구,1141011400,신촌동,13,00,1,704.0
1,20230101,11,서울특별시,11110,종로구,1111010100,청운동,11,서울특별시,11440,마포구,1144012000,서교동,15,00,1,3688.0
2,20230101,11,서울특별시,11110,종로구,1111010100,청운동,11,서울특별시,11440,마포구,1144012000,서교동,17,00,1,1227.0
3,20230101,11,서울특별시,11110,종로구,1111010100,청운동,11,서울특별시,11650,서초구,1165010800,서초동,08,15,2,2204.0
4,20230101,11,서울특별시,11110,종로구,1111010100,청운동,11,서울특별시,11650,서초구,1165010800,서초동,08,30,1,2407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50268,20230101,11,서울특별시,11740,강동구,1174011000,강일동,11,서울특별시,11680,강남구,1168011800,도곡동,08,45,1,3231.0
50269,20230101,11,서울특별시,11740,강동구,1174011000,강일동,11,서울특별시,11680,강남구,1168011800,도곡동,11,30,1,3447.0
50270,20230101,11,서울특별시,11740,강동구,1174011000,강일동,11,서울특별시,11680,강남구,1168011800,도곡동,12,00,1,3516.0
50271,20230101,11,서울특별시,11740,강동구,1174011000,강일동,11,서울특별시,11680,강남구,1168011800,도곡동,13,00,1,3565.0


### 메인코드2(백업 코드 추가)

In [46]:
def load_progress():
    try:
        with open('progress.json', 'r') as f:
            data = json.load(f)
            return data['date_idx'], data['i'], data['j'], data['direction']
    except FileNotFoundError:
        return 0, 0, 0, 0  # date_idx, i, j, direction

def save_progress(date_idx, i, j, direction):
    with open('progress.json', 'w') as f:
        json.dump({'date_idx': date_idx, 'i': i, 'j': j, 'direction': direction}, f)

def save_data(all_results):
    df = pd.DataFrame(all_results)
    df.to_csv('data.csv', index=False)


In [48]:
################ 수정 구간 ################
API_KEY = "20250516163905aa0363qf6pse5eedgu7j5n111m"
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 1)
################ 수정 구간 ################

# 날짜 리스트 생성
date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date)
    current_date += timedelta(days=1)

# 진행상황 불러오기
date_idx, i, j, direction = load_progress()
all_results = []
save_interval = 20  # 몇 번마다 저장할지(적당히 조절)

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            try:
                data = await response.json()
                return data.get('result', [])
            except Exception as e:
                print(f"[JSON 파싱 오류] {url}\n{e}")
                return []
        else:
            print(f"Error: {response.status} / {url}")
            return []

async def main():
    global all_results
    count = 0
    async with aiohttp.ClientSession() as session:
        for d_idx in range(date_idx, len(date_list)):
            opratDate = date_list[d_idx].strftime('%Y%m%d')
            # direction == 0: 서울→핫플, direction == 1: 핫플→서울
            dir_start = direction if d_idx == date_idx else 0
            for dir in range(dir_start, 2):
                cd1 = seoul_cd_list if dir == 0 else hot_seoul_cd_list
                cd2 = hot_seoul_cd_list if dir == 0 else seoul_cd_list
                i_start = i if (d_idx == date_idx and dir == direction) else 0
                for i_idx in range(i_start, len(cd1)):
                    j_start = j if (d_idx == date_idx and dir == direction and i_idx == i) else 0
                    for j_idx in range(j_start, len(cd2)):
                        stgEmdCd = cd1[i_idx]
                        arrEmdCd = cd2[j_idx]
                        if stgEmdCd == arrEmdCd:
                            continue
                        url = (
                            f"https://stcis.go.kr/openapi/quarterod.json"
                            f"?apikey={API_KEY}"
                            f"&opratDate={opratDate}"
                            f"&stgEmdCd={stgEmdCd}"
                            f"&arrEmdCd={arrEmdCd}"
                        )
                        result = await fetch(session, url)
                        all_results.extend(result)
                        count += 1
                        # 중간 저장
                        if count % save_interval == 0:
                            save_data(all_results)
                            save_progress(d_idx, i_idx, j_idx, dir)
                            print(f"진행상황 저장: 날짜 idx={d_idx}, i={i_idx}, j={j_idx}, direction={dir}, 누적 {len(all_results)}건")
            # direction, i, j는 다음 날짜부터 0으로 자동 초기화됨
        # 마지막 저장
        save_data(all_results)
        save_progress(len(date_list), 0, 0, 0)
        print("최종 저장 완료")

await main()
print(f"총 데이터 건수: {len(all_results)}")


Error: 502 / https://stcis.go.kr/openapi/quarterod.json?apikey=20250516163905aa0363qf6pse5eedgu7j5n111m&opratDate=20230101&stgEmdCd=1111010100&arrEmdCd=1111016500
Error: 502 / https://stcis.go.kr/openapi/quarterod.json?apikey=20250516163905aa0363qf6pse5eedgu7j5n111m&opratDate=20230101&stgEmdCd=1111010100&arrEmdCd=1121510200
Error: 502 / https://stcis.go.kr/openapi/quarterod.json?apikey=20250516163905aa0363qf6pse5eedgu7j5n111m&opratDate=20230101&stgEmdCd=1111010100&arrEmdCd=1141011400
Error: 502 / https://stcis.go.kr/openapi/quarterod.json?apikey=20250516163905aa0363qf6pse5eedgu7j5n111m&opratDate=20230101&stgEmdCd=1111010100&arrEmdCd=1144012000
Error: 502 / https://stcis.go.kr/openapi/quarterod.json?apikey=20250516163905aa0363qf6pse5eedgu7j5n111m&opratDate=20230101&stgEmdCd=1111010100&arrEmdCd=1165010800
Error: 502 / https://stcis.go.kr/openapi/quarterod.json?apikey=20250516163905aa0363qf6pse5eedgu7j5n111m&opratDate=20230101&stgEmdCd=1111010100&arrEmdCd=1168010100
Error: 502 / https://s

CancelledError: 

### 수정 받은 코드

In [ ]:
# API 입력
API_KEY_LIST = ['20250518005638uc0bmkthkfrna9rv81ehcjq1vk', '2025051800484186vse5j701jk46q2ptnkghm1kd', '202505180048567unbcjmirodnlq96krutt3l93e', '20250518004909p4ae0tb3eepur835j68mn33ucm', '20250518005128o53ivgo6dn0j4q4e3fpq0ltta1']
KEY_INDEX = 0

In [14]:
#저장 경로 지정
DATA_CSV_PATH = 'seoul_OD_1.csv'
PROGRESS_JSON_PATH = 'seoul_OD_progress_1.json'

In [ ]:
#날짜
start_date = datetime(2023, 1, 1) # 시작 날짜
end_date = datetime(2023, 12, 31) # 종료 날짜

In [ ]:
def get_api_key():
    global KEY_INDEX
    key = API_KEY_LIST[KEY_INDEX]
    KEY_INDEX = (KEY_INDEX + 1) % len(API_KEY_LIST)
    return key

def save_progress(i, j, k):
    with open(PROGRESS_JSON_PATH, 'w') as f:
        json.dump({'i': i, 'j': j, 'k': k}, f)

def load_progress():
    try:
        with open(PROGRESS_JSON_PATH, 'r') as f:
            data = json.load(f)
            return data['i'], data['j'], data['k']
    except FileNotFoundError:
        return 0, 0, 0

def save_data(df):
    df.to_csv(DATA_CSV_PATH, index=False)

def load_data(filename=DATA_CSV_PATH):
    if not os.path.exists(filename) or os.stat(filename).st_size == 0:
        return pd.DataFrame()
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return pd.DataFrame()

In [ ]:
async def fetch(session, url):
    try:
        async with session.get(url, timeout=20) as response:
            if response.status == 200:
                data = await response.json()
                if data.get('status') == 'OK':
                    return data.get('result', [])
                else:
                    return []
            else:
                print(f"Error: {response.status} / {url}")
                return []
    except Exception as e:
        print(f"Request Exception: {e} / {url}")
        return []

async def main():
    all_df = load_data()
    i_start, j_start, k_start = load_progress()

    dates = []
    cur_date = start_date
    while cur_date <= end_date:
        dates.append(cur_date)
        cur_date += timedelta(days=1)

    total_iterations = len(dates) * (len(seoul_cd_list) * len(hot_seoul_cd_list)) * 2
    # 진행상황 맞게 재설정 - 실제 반복은 출발지와 도착지 리스트 합쳐서 처리하므로 단순화하긴 어려움. 대략으로 진행바 설정
    pbar = tqdm.tqdm(total=total_iterations, desc="전체 진행", dynamic_ncols=True)
    # 진행상황 초기화
    # 아래에서 진행상황 따라 업데이트 할 예정
    async with aiohttp.ClientSession() as session:
        for i in range(i_start, len(dates)):
            opratDate = dates[i].strftime('%Y%m%d')

            # 1. 출발지: 서울시, 도착지: 핫플레이스
            for j in range(j_start if i == i_start else 0, len(seoul_cd_list)):
                stgEmdCd = seoul_cd_list[j]
                for k in range(k_start if i == i_start and j == j_start else 0, len(hot_seoul_cd_list)):
                    arrEmdCd = hot_seoul_cd_list[k]
                    if stgEmdCd == arrEmdCd:
                        pbar.update(1)
                        continue
                    key = get_api_key()
                    url = (
                        f"https://stcis.go.kr/openapi/quarterod.json"
                        f"?apikey={key}"
                        f"&opratDate={opratDate}"
                        f"&stgEmdCd={stgEmdCd}"
                        f"&arrEmdCd={arrEmdCd}"
                    )
                    result = await fetch(session, url)
                    if result:
                        daily_df = pd.DataFrame(result)
                        all_df = pd.concat([all_df, daily_df], ignore_index=True)
                        save_data(all_df)
                    save_progress(i, j, k)
                    pbar.update(1)
                    await asyncio.sleep(0.5)  # API 부하 방지

            # 2. 출발지: 핫플레이스, 도착지: 서울시
            for j in range(j_start if i == i_start else 0, len(hot_seoul_cd_list)):
                stgEmdCd = hot_seoul_cd_list[j]
                for k in range(k_start if i == i_start and j == j_start else 0, len(seoul_cd_list)):
                    arrEmdCd = seoul_cd_list[k]
                    if stgEmdCd == arrEmdCd:
                        pbar.update(1)
                        continue
                    key = get_api_key()
                    url = (
                        f"https://stcis.go.kr/openapi/quarterod.json"
                        f"?apikey={key}"
                        f"&opratDate={opratDate}"
                        f"&stgEmdCd={stgEmdCd}"
                        f"&arrEmdCd={arrEmdCd}"
                    )
                    result = await fetch(session, url)
                    if result:
                        daily_df = pd.DataFrame(result)
                        all_df = pd.concat([all_df, daily_df], ignore_index=True)
                        save_data(all_df)
                    save_progress(i, j, k)
                    pbar.update(1)
                    await asyncio.sleep(0.5)  # API 부하 방지

            # 날짜 루프 종료 후 j_start, k_start 초기화
            j_start = 0
            k_start = 0

    pbar.close()
    print(f"✅ 모든 데이터 수집 완료! 총 건수: {len(all_df)}")

# Jupyter 환경에서는 아래처럼 호출
# await main()

# 필요 컬럼만 추출

In [106]:
df_for_pivot = df[['opratDate','stgSdNm','stgSggNm','stgEmdNm','arrSdNm','arrSggNm','arrEmdNm','useStf']]
df_for_pivot.head()

,opratDate,stgSdNm,stgSggNm,stgEmdNm,arrSdNm,arrSggNm,arrEmdNm,useStf
0,20230101,서울특별시,종로구,청운동,서울특별시,서대문구,신촌동,1
1,20230101,서울특별시,종로구,청운동,서울특별시,마포구,서교동,1
2,20230101,서울특별시,종로구,청운동,서울특별시,마포구,서교동,1
3,20230101,서울특별시,종로구,청운동,서울특별시,서초구,서초동,2
4,20230101,서울특별시,종로구,청운동,서울특별시,서초구,서초동,1


In [ ]:
df_for_pivot.pivot_table(
    index='opratDate',
    columns='arrEmdNm',
    values='useStf',
    aggfunc='sum')

arrEmdNm,능동,도곡동,삼성동,서교동,서초동,수서동,신촌동,역삼동,이화동
opratDate,,,,,,,,,
20230101,6106,13597,39068,24980,49219,13858,1256,58030,256
